In [1]:
import os 
import json 
import pandas as pd
import traceback

In [5]:

import langchain
from langchain.chat_models import ChatOpenAI
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent, create_react_agent

In [9]:
from dotenv import load_dotenv
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [10]:
print(openai_key)

sk-proj-7vq7dr4YgAlK9iqPO5aBT3BlbkFJMYLt45TTP6HVHtg5za5c


In [4]:
#environmemnt


import os
os.environ['HUGGINGFACEHUB_API_TOKEN']=HUGGINGFACEHUB_API_TOKEN 


In [3]:


question = "Who won the FIFA World Cup in the year 2022? "

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

repo_id =  "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_length=128, temperature=0.5, token=HUGGINGFACEHUB_API_TOKEN
)
llm_chain = LLMChain(prompt=prompt, llm=llm)
print(llm_chain.run(question))

NameError: name 'HUGGINGFACEHUB_API_TOKEN' is not defined